<a href="https://colab.research.google.com/github/kshitij9969/Indicator_of_sucess_dataset/blob/main/Indicator_of_sucess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# To do: 
# Two types of data needs to be extracted
# 1. Indexes
# 2. True values

# 1. Cost of living index by city
# 2. Cost of living index by country
# 3. Property price index
# 4. Quality of life index
# 5. Crime index
# 6. Health care index
# 7. Pollution index
# 8. Traffic index
# 9. 


# 3. Cost of living estimator by city
# 4. Food prices by city
# 5. Food prices by country
# 6. Prices of other commodity by city and country
# 7. Basket of prices by city
# 8. Taxi fare by city
# 9. Gas prices by city


# Cost of living index

### Cost of living index over the years. 

In [ ]:
response = requests.get("https://www.numbeo.com/cost-of-living/rankings.jsp")

In [ ]:
df = pd.read_html(response.text)

In [ ]:
df

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
table = soup.find("table", attrs={"id": "t2"})

In [ ]:
df = pd.read_html(str(table))[0]

In [ ]:
# Cleaning columns

for col in df.columns:
    df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)


In [ ]:
# Split city and country

df['Country'] = df['City'].apply(lambda x: x.split(", ")[-1].strip())

In [ ]:
df['City'] = df['City'].apply(lambda x: x.split(", ")[0].strip())

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['Rank'] = df['index']

In [ ]:
df.drop(columns=['index'], inplace=True)

In [ ]:
df['Rank'] = df['Rank'].apply(lambda x: x + 1)

In [ ]:
df['Year'] = 2021
df['Month'] = 6
df['Day'] = 1

In [ ]:
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [ ]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', '2011', '2010', '2009']

for (i, continent) in links:
    res = requests.get(i)
    soup = BeautifulSoup(res.text)
    try:
        spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
    except Exception as e:
        # Oceania doesn't have any regions
        print(e)
        print(i)
    print(spans)
    for span in spans:
        url = span.find("a", href=True)['href']
        resp = requests.get(url)
        soup1 = BeautifulSoup(resp.text)
        # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
        # print(resp.text)
        table = soup.find("table", attrs={"id": "t2"})
        df = pd.read_html(str(table))[0]
        # Clean columns
        for col in df.columns:
            df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
        
        # Correct Rank
        df.reset_index(inplace=True)
        df['Rank'] = df['index']
        df['Rank'] = df['Rank'].apply(lambda x: x + 1)
        df.drop(columns=['index'], inplace=True)
        # Add month year and day
        df['Year'] = 2021
        df['Month'] = 1
        df['Day'] = 1

        df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
        # Add country and continent
        df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
        df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
        df['Continent'] = continent
        df['Region'] = soup1.find("h1").text.split(":")[0].strip()
        print(url)
        print(years_list)
    break
        # print(url)
        # print(years_list)
        # region_name = soup1.find("h1").text.split(":")[0].strip()
        # print(region_name)






In [ ]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', 
              '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', 
              '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', 
              '2011', '2010', '2009']
url_list = []

for year in years_list:
    for (i, continent) in links:
        res = requests.get(i.replace("year", year))
        soup = BeautifulSoup(res.text)
        try:
            spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
        except Exception as e:
            # Oceania doesn't have any regions
            print(e)
            print(i)
        for span in spans:
            url = span.find("a", href=True)['href']
            url_list.append(url)
print(url_list)




In [ ]:
url_list

In [ ]:
final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date', 'URL'])
for url in url_list:
    resp = requests.get(url)
    print(url)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    df['Continent'] = continent
    df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df)
    final_dataframe = final_dataframe.append(df)



In [ ]:
final_dataframe[final_dataframe.Year == 2015]

In [ ]:
# Correcting the Continent


def get_continent(x):
    if "Africa" in x:
        return "Africa"
    elif "America" in x:
        return "America"
    elif "Oceania" in x:
        return "Oceania"
    elif "Asia" in x:
        return "Asia"
    else:
        return "Europe"

final_dataframe['Continent'] = final_dataframe['Region'].apply(lambda x: get_continent(x))

In [ ]:
final_dataframe[(final_dataframe.City == "Aachen") & (final_dataframe.duplicated() == True)].sort_values(by="City")

In [ ]:
final_dataframe.reset_index(drop=True, inplace=True)

In [ ]:
final_dataframe.drop_duplicates(subset=['City', 'Country', 'Continent', 'Region', 'Year', 'Month', 'Day', 'Date'])

In [ ]:
final_dataframe.to_csv("Cost_of_living_index.csv", index=False)

In [ ]:
property_url_list = [i.replace("cost-of-living", "property-investment") for i in url_list]

In [ ]:
# Property index

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Price_To_Income_Ratio',
                                        'Gross_Rental_Yield_City_Centre',
                                        'Gross_Rental_Yield_Outside_of_Centre',
                                        'Price_To_Rent_Ratio_City_Centre',
                                        'Price_To_Rent_Ratio_Outside_Of_City_Centre',
                                        'Mortgage_As_A_Percentage_Of_Income',
                                        'Affordability_Index', 'Year', 
                                        'Month', 'Day', 'Date', 'URL'])
for url in property_url_list:
    resp = requests.get(url)
    print(url)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    df['Continent'] = continent
    df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)







In [ ]:
final_dataframe['Continent'] = final_dataframe.Region.apply(lambda x: get_continent(x))

In [ ]:
final_dataframe.to_csv("Property_price_index.csv", index=False)

In [ ]:
final_dataframe

In [ ]:
# Property price index by country

# Property index

final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Price_To_Income_Ratio',
                                        'Gross_Rental_Yield_City_Centre',
                                        'Gross_Rental_Yield_Outside_of_Centre',
                                        'Price_To_Rent_Ratio_City_Centre',
                                        'Price_To_Rent_Ratio_Outside_Of_City_Centre',
                                        'Mortgage_As_A_Percentage_Of_Income',
                                        'Affordability_Index', 'Year', 
                                        'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/property-investment/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    print(df)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)


In [ ]:
final_dataframe.to_csv("Property_price_index_by_country.csv", index=False)

In [ ]:
# Quality of life index by country and city

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Quality_of_Life_Index', 
                                        'Purchasing_Power_Index', 
                                        'Safety_Index', 'Health_Care_Index', 
                                        'Cost_of_Living_Index', 
                                        'Property_Price_to_Income_Ratio', 
                                        'Traffic_Commute_Time_Index', 
                                        'Pollution_Index', 'Climate_Index', 
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/quality-of-life/rankings.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    print(df.columns)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)




In [ ]:
final_dataframe.to_csv("Quality_of_life_index_by_city.csv", index=False)

In [ ]:
# Quality of life index by country

final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Quality_of_Life_Index', 
                                        'Purchasing_Power_Index', 
                                        'Safety_Index', 'Health_Care_Index', 
                                        'Cost_of_Living_Index', 
                                        'Property_Price_to_Income_Ratio', 
                                        'Traffic_Commute_Time_Index', 
                                        'Pollution_Index', 'Climate_Index', 
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    print(df.columns)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)





In [ ]:
final_dataframe.to_csv("Quality_of_life_index_by_country.csv", index=False)

In [ ]:
# Crime index by country and city

# Quality of life index by country and city

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Crime_Index', 'Safety_Index', 
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/crime/rankings.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)






In [ ]:
final_dataframe.to_csv("Crime_index_by_city.csv", index=False)

In [ ]:
# Crime index by country

# Quality of life index by country

final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Crime_Index', 'Safety_Index', 
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/crime/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    print(df.columns)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)



In [ ]:
final_dataframe.to_csv("Crime_index_by_country.csv", index=False)

In [ ]:
# Health care index by country and city

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Health_Care_Index', 'Health_CareExp_Index',
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/health-care/rankings.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)






In [ ]:
final_dataframe.to_csv("Health_care_index_by_city.csv", index=False)

In [ ]:
# Health index by country

final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Health_Care_Index', 'Health_CareExp_Index',
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/health-care/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    print(df.columns)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)



In [ ]:
final_dataframe.to_csv("Health_index_by_country.csv", index=False)

In [ ]:
# Pollution index by country and city


final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Pollution_Index', 'Exp_Pollution_Index',
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/pollution/rankings.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)




In [ ]:
final_dataframe.to_csv("Pollution_index_by_city.csv", index=False)

In [ ]:
# Pollution index by country

final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Pollution_Index', 'Exp_Pollution_Index',
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/pollution/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    print(df.columns)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)






In [ ]:
final_dataframe.to_csv("Pollution_index_by_country.csv", index=False)

In [29]:
import os
import shutil
files=os.listdir("/content/")

def save_to_drive():
    # iterating over all the files in
    # the source directory
    for fname in files:
        if ".csv" in fname:
            shutil.copy2(os.path.join("/content/",fname), "/content/drive/MyDrive/Indicator_of_success_dataset/")

In [ ]:
# Traffic index by country and city


final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Traffic_Index', 'Time_Index_in_minutes',
                                        'Time_Exp_Index', 'Inefficiency_Index', 'CO2_Emission_Index', 
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/traffic/rankings.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    
    df.rename(columns={"Time_Index(in_minutes)": "Time_Index_in_minutes"}, inplace=True)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)






In [ ]:
final_dataframe.to_csv("Traffic_index_by_city.csv", index=False)

In [ ]:
# Traffic index by country


final_dataframe = pd.DataFrame(columns=['Rank', 'Country', 
                                        'Traffic_Index', 'Time_Index_in_minutes',
                                        'Time_Exp_Index', 'Inefficiency_Index', 'CO2_Emission_Index',
                                        'Year', 'Month', 'Day', 'Date', 'URL'])

url = "https://www.numbeo.com/traffic/rankings_by_country.jsp?title="
for year in years_list:
    resp = requests.get(url + year)
    print(url + year)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # print(df.columns)
    # break
    # print(table)
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join([i.replace(".", "") for i in col.split(" ")])}, inplace=True)
    print(df.columns)
    df.rename(columns={"Time_Index(in_minutes)": "Time_Index_in_minutes"}, inplace=True)
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    # year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Continent'] = continent
    # df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df, df.columns, df.loc[0])
    # break
    final_dataframe = final_dataframe.append(df)






In [ ]:
final_dataframe.to_csv("Traffic_index_by_country.csv", index=False)

## Cost of living true values by country

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.numbeo.com/cost-of-living/")

soup2 = BeautifulSoup(response.text)

tds = soup2.find("table", attrs={"class": "related_links"}).find("tr").find_all("td")

a = [td.find_all("a", href=True) for td in tds]

links = []

for i in a:
    for j in i:
        links.append([j['href'], j.text])



In [2]:
final_dataframe = pd.DataFrame(columns=['Commodity', 'Price', 'Lower_Range', 'Upper_Range'])
for [link, country] in links:
    response = requests.get("https://www.numbeo.com/cost-of-living/" + link)
    soup3 = BeautifulSoup(response.text)
    table = soup3.find("table", attrs={"class": "data_wide_table new_bar_table"})
    df = pd.read_html(str(table))[0]
    df['Country'] = country
    df = df[df['Edit'] != 'Edit']
    df.rename(columns={"Restaurants": "Commodity", "Edit": "Price"}, inplace=True)
    # df['Currency'] = df['Price'].apply(lambda x: str(x.split(" ")[-1].strip()) if str(x) != 'nan' else x)
    # df['Price'] = df['Price'].apply(lambda x: float(x.split(" ")[0].strip()) if str(x) != 'nan' else x)
    df['Lower_Range'] = df['Range'].apply(lambda x: float(str(x).split("-")[0].strip()) if str(x) != 'nan' else x)
    df['Upper_Range'] = df['Range'].apply(lambda x: float(str(x).split("-")[1].strip()) if str(x) != 'nan' else x)
    final_dataframe = final_dataframe.append(df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] ==

In [ ]:
len(final_dataframe.Country.unique())

In [4]:
final_dataframe.to_csv("Cost_of_living_by_country.csv", index=False)

In [74]:
# Food prices by country

import re

final_dataframe = pd.DataFrame(columns=["Commodity", "Country", "Quantity", "Unit", "Price"])

for link in links:
    response = requests.get("https://www.numbeo.com/food-prices/" + link[0])
    soup4 = BeautifulSoup(response.text)
    table = soup4.find_all("table")[1]
    df = pd.read_html(str(table))[0]
    df = df.iloc[:-2]
    df.rename(columns={0: "Commodity", 1: "Price"}, inplace=True)
    df['Quantity'] = df["Commodity"].apply(lambda x: re.findall(r"\([a-zA-Z0-9(\.)+ ]+\)$", x.strip())[-1])
    df['Quantity'] = df['Quantity'].apply(lambda x: x.split(") ("))
    df['Quantity'] = df['Quantity'].apply(lambda x: [i for i in x if bool(re.search("[0-9]+", i))])
    df['Quantity'] = df['Quantity'].apply(lambda x: x[0].strip("(").strip(")"))
    df['Unit'] =df['Quantity'].apply(lambda x: x.split(" ")[-1] if len(x.split(" ")) > 1 else np.nan)
    df['Quantity'] =df['Quantity'].apply(lambda x: x.split(" ")[0] if len(x.split(" ")) > 1 else x)
    df['Country'] = link[-1]
    # df['Quantity'] = df['Quantity'].apply(lambda x: x.split(","))
    final_dataframe = final_dataframe.append(df)
    # break



In [75]:
final_dataframe

,Commodity,Country,Quantity,Unit,Price
0,"Milk (regular), (0.25 liter)",Afghanistan,0.25,liter,13.03 AFN
1,Loaf of Fresh White Bread (125.00 g),Afghanistan,125.00,g,8.04 AFN
2,"Rice (white), (0.10 kg)",Afghanistan,0.10,kg,9.33 AFN
3,Eggs (regular) (2.40),Afghanistan,2.40,NaN,20.68 AFN
4,Local Cheese (0.10 kg),Afghanistan,0.10,kg,31.49 AFN
...,...,...,...,...,...
9,Oranges (0.30 kg),Zimbabwe,0.30,kg,0.59 $
10,Tomato (0.20 kg),Zimbabwe,0.20,kg,0.32 $
11,Potato (0.20 kg),Zimbabwe,0.20,kg,0.36 $
12,Onion (0.10 kg),Zimbabwe,0.10,kg,0.16 $


In [76]:
final_dataframe.to_csv("Food_prices_by_country.csv", index=False)
save_to_drive()

In [66]:
len("2.40".split(" "))

1

In [77]:
# Prices of other commodities by country and city

url = "https://www.numbeo.com/cost-of-living/prices_by_city.jsp"

# Get all the item ids

response = requests.get(url)

soup5 = BeautifulSoup(response.text)

option = soup5.find("table", attrs={"class": "table_for_selecting_items"}).find_all("tr")[1].find_all("td")[-1].find("select").find_all("option")



item_ids = [i['value'] for i in option]





In [90]:
final_dataframe = pd.DataFrame()
url = f"https://www.numbeo.com/cost-of-living/prices_by_city.jsp?displayCurrency=USD&itemId={item_id}"
response = requests.get(url)
soup6 = BeautifulSoup(response.text)
table = soup6.find("table", attrs={"id": "t2"})
final_dataframe = pd.read_html(str(table))[0]
# final_dataframe['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
# final_dataframe['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
# final_dataframe['Currency'] = "$"
final_dataframe.drop(columns=['Rank'], inplace=True)
for item_id in item_ids[1:]:
    url = f"https://www.numbeo.com/cost-of-living/prices_by_city.jsp?displayCurrency=USD&itemId={item_id}"
    response = requests.get(url)
    soup6 = BeautifulSoup(response.text)
    table = soup6.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    df.drop(columns="Rank", inplace=True)
    # df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    # df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    # df['Currency'] = "$"
    final_dataframe = pd.merge(final_dataframe, df, on="City", how="outer")

In [93]:
final_dataframe['Country'] = final_dataframe['City'].apply(lambda x: x.split(",")[-1].strip())
final_dataframe['City'] = final_dataframe['City'].apply(lambda x: x.split(",")[0].strip())
final_dataframe['Currency'] = "$"
final_dataframe.to_csv("Price_of_other_commodities_by_city.csv", index=False)
save_to_drive()

In [4]:
# Taxi fare by country

import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.numbeo.com/taxi-fare/")

soup7 = BeautifulSoup(response.text)
tds = soup7.find("table", attrs={"class": "related_links"}).find("tr").find_all("td")

links = []

for td in tds:
    links.extend([(a['href'], a.text) for a in td.find_all("a", href=True)])




In [15]:
import pandas as pd
final_dataframe = pd.DataFrame(columns=['City', 'Taxi Start(Normal Tariff)', 'Taxi 1km(Normal Tariff)',
       'Taxi 1hour waiting(Normal Tariff)'])
for link in links:
    url = "https://www.numbeo.com/taxi-fare/"
    response = requests.get(url + link[0])
    table = BeautifulSoup(response.text).find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    final_dataframe = final_dataframe.append(df)

In [16]:
final_dataframe['Currency'] = final_dataframe['Taxi Start(Normal Tariff)'].apply(lambda x: x.split(" ")[-1].strip())
# for col in final_dataframe.columns:
#     if col != "City":
#         final_dataframe[col] = final_dataframe[col].apply(lambda x: float(x.replace(u'\xa0', u' ').split(" ")[0].strip().replace(",", "")))

In [25]:
final_dataframe['Currency'] = final_dataframe['Taxi Start(Normal Tariff)'].apply(lambda x: str(x).split(" ")[-1])

In [30]:
final_dataframe.to_csv("Taxi_fare_by_city.csv", index=False)
save_to_drive()

In [27]:
final_dataframe.dtypes

City                                 object
Taxi Start(Normal Tariff)            object
Taxi 1km(Normal Tariff)              object
Taxi 1hour waiting(Normal Tariff)    object
Currency                             object
dtype: object